In [2]:
#!pip install torch transformers numpy
import torch
from transformers import GPT2Model, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2Model.from_pretrained("gpt2")
model.eval()
print("konfiguracja modelu:", model.config)


konfiguracja modelu: GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "dtype": "float32",
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.57.3",
  "use_cache": true,
  "vocab_size": 50257
}



<center><h1>Macierz embedding</h1></center>

Macierz embedding oznaczamy jako $W_E$ . Model GPT2 ma predefiniowany słownik składający się z $V = 50\,257$ tokenów. 

Dla każdego z tych tokenów odpowiada jedna kolumna tej macierzy. Każda kolumna (w przypadku modelu GPT2-small) stanowi 768-wymiarowy wektor cech. Oznacza to, że macierz embedding ma wymiary $768 \times50\,257$. Na podstawie tego możemy obliczyć wartość wszystkich parametrów w macierzy embedding.

In [17]:
we = model.wte.weight
print("kształt macierzy embedding:", we.shape)

kolumny = we.shape[1]
wiersze = we.shape[0]
parametry = kolumny*wiersze
print("wszystkie parametry w macierzy embedding:", parametry)

kształt macierzy embedding: torch.Size([50257, 768])
wszystkie parametry w macierzy embedding: 38597376


<center><h1>Macierze Q,K,V</h1></center>

W bloku self attention wykorzystujemy trzy macierze wag, które oznaczamy jako $W_Q$ (Query), $W_K$ (Key), $W_V$ (Value). W modelu GPT2-small ich wymiary wynoszą $768 \times768$, choć technicznie są one podzielone na niezależne podprzestrzenie zwane głowicami.

Model posiada 12 głowic. Oznacza to, że obliczony wektor zapytania $q$  o długości 768 w rzeczywistości składa się z 12 mniejszych wektorów, każdy o wymiarze:

<center><h4>$d_{head} = \frac{768}{12} = 64$</h4></center>

* **wiersze** odpowiadają wymiarowi wejściowemu – macierz musi przyjąć wektor embeddingu o długości 768.
* **kolumny** odpowiadają wymiarowi wyjściowemu – wynikiem mnożenia jest nowy wektor, który również ma długość 768.

Mnożąc wektor konkretnego tokena przez macierz $W_Q$ otrzymujemy wektor zapytania $q$, który szuka konkretnych cech

Mnożąc wektor konkretnego tokena przez macierz $W_K$ otrzymujemy wektor klucza $k$, który reklamuje konkretne cechy

Mnożąc wektor konkretnego tokena przez macierz $W_V$ otrzymujemy wektor wartości $v$, który zawiera treść, która zostanie przekazana dalej, jeśli $q$ i $k$ będą do siebie pasować.


In [6]:
e = model.wte.weight[100] 

W_QKV = model.h[0].attn.c_attn.weight

W_Q = W_QKV[:, 0:768]
W_K = W_QKV[:, 768:1536]
W_V = W_QKV[:, 1536:]
print("kształt W_Q:", W_Q.shape)
print("kształt W_K:", W_K.shape)
print("kształt W_V:", W_V.shape)

# wektor tokena*macierz
q = torch.matmul(e, W_Q)
k = torch.matmul(e, W_K)
v = torch.matmul(e, W_V)
print("")
print("wektor q:", q.shape)
print("wektor k:", k.shape)
print("wektor v:", v.shape)

n_head = model.config.n_head
head_dim = q.shape[0] // n_head
print("")
print("liczba głowic w modelu:", n_head)
print("rzeczywisty wymiar wektora jednej głowicy:", head_dim)

kształt W_Q: torch.Size([768, 768])
kształt W_K: torch.Size([768, 768])
kształt W_V: torch.Size([768, 768])

wektor q: torch.Size([768])
wektor k: torch.Size([768])
wektor v: torch.Size([768])

liczba głowic w modelu: 12
rzeczywisty wymiar wektora jednej głowicy: 64
